In [ ]:
# Import external tools:
import time 
import random
from matplotlib import pyplot as plt
import librosa
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torch.utils.data.sampler import SubsetRandomSampler
from torchsummary import summary
import torchaudio
import soundfile as sf
from scipy import signal
import getpass
import pandas as pd
import numpy as np
import sys
import os
import importlib
from scipy.io import wavfile
from IPython.display import Audio
from datetime import datetime
from sklearn.manifold import TSNE

# I am running this script on two different computers, so i need to change paths
# depending on computer in use: 
if getpass.getuser()=="joanna.luberadzka":
    projectdir="/Users/joanna.luberadzka/Documents/VAE-IR/"
    datadir="/Users/joanna.luberadzka/Documents/Data/IR_Arni_upload_numClosed_0-5/"
elif getpass.getuser()=="ubuntu":
    projectdir="/home/ubuntu/joanna/VAE-IR/"
    datadir="/home/ubuntu/Data/IR_Arni_upload_numClosed_0-5/"

# Add path of this project
sys.path.insert(0, projectdir+'src/')

# Import and automatically reload my own modules:
import models; importlib.reload(models)
import train; importlib.reload(train)
import datasetprep as dsprep; importlib.reload(dsprep)
import helpers; importlib.reload(helpers)

In [ ]:
# I created a table summarizing two IR data bases (Arni and BUT)
# Table contains file path, and acoustic parameters: rt,drr,cte,edt (computed using matlab toolbox)

# -- Data: --
if getpass.getuser()=="joanna.luberadzka":
    INFO_FILE = projectdir + "irstats_ARNIandBUT_local.csv"
elif getpass.getuser()=="ubuntu":
    INFO_FILE = projectdir+"irstats_ARNIandBUT_datura.csv"

IrData = pd.read_csv(INFO_FILE,delimiter=',')
IrData.head()

In [ ]:
# Each entry of that database represents one recorded room impulse response. 
# Below, I plot a random sample from the database and print its acoustic parameters computed with a Matlab toolbox.

rand_ir_idx=random.sample(range(len(IrData)),1)[0]
# get info of an impulse response with a specific index
print(f"The filename of RIR sample {rand_ir_idx} is: "+ IrData["filepath"][rand_ir_idx])
print("The reverberation time is: rt= "+ str(IrData["rt"][rand_ir_idx]))
print("The direct-to-reverberant ratio is: drr= "+ str(IrData["drr"][rand_ir_idx]))
print("The early-to-late reflections ratio is: cte= "+ str(IrData["cte"][rand_ir_idx]))
print("The early decay time is: edt= "+ str(IrData["drr"][rand_ir_idx]))

# get rir waveform and spectrum
h,H,_= helpers.wav2powspec(IrData["filepath"][rand_ir_idx])

# plot waveform and spectrogram
plt.figure(figsize=(10,2))
plt.subplot (1,2,1)
plt.imshow(librosa.power_to_db(H), origin="lower", aspect="auto")
plt.title(f'Log-spectrogram of RIR sample {rand_ir_idx}')
plt.subplot (1,2,2)
plt.plot(h)
plt.title(f'Waveform of RIR sample {rand_ir_idx}')
plt.show()

In [ ]:
# In the scatterplots below each point represents one impulse response. I plotted rt as a function of drr and add a colormap 
# to represent additional information about each ir (which database, cte, edt). 

# plot parameters of impulse responses in the data  
fig, ax = plt.subplots()
ax.scatter(x=IrData["rt"][0:11331], y=IrData["drr"][0:11331],label="Arni", alpha=0.2)
ax.scatter(x=IrData["rt"][11331:], y=IrData["drr"][11331:],label="BUT",alpha=0.2)
plt.xlabel("T60")
plt.ylabel("DRR")
plt.legend()
ax.set(title="Colormap: Two IR data bases")
plt.show()

fig, ax = plt.subplots()
ax.scatter(x=IrData["rt"], y=IrData["drr"],c=IrData["cte"], alpha=0.2)
sc1=plt.xlabel("T60")
sc2=plt.ylabel("DRR")
ax.set(title="Colormap: Early to late reflections (CTE) ")
plt.show()

fig, ax = plt.subplots()
ax.scatter(x=IrData["rt"], y=IrData["drr"],c=IrData["edt"], alpha=0.2)
plt.xlabel("T60")
plt.ylabel("DRR")
ax.set(title="Colormap: Early decay time (EDT)")
plt.show()

In [ ]:
# I want to use the IR database and train an autoencoder to learn the transformation between the 
# input impulse response and a lower-dimensional embedding space. 

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps")

# ----- DATA: -----
if getpass.getuser()=="joanna.luberadzka":
    INFO_FILE = projectdir + "irstats_ARNIandBUT_datura.csv"
elif getpass.getuser()=="ubuntu":
    INFO_FILE = projectdir+"irstats_ARNIandBUT_datura.csv"

# Create dataset object
SAMPLING_RATE=8e3
PREPROC="powspec"
dataset = dsprep.DatasetRirs(INFO_FILE,SAMPLING_RATE,PREPROC)


# how much data should be used from the data base
perc_data_use=1
N_used = round(len(dataset) * perc_data_use)
N_unused = len(dataset) - N_used
usedset, unusedset = random_split(dataset, [N_used, N_unused])
# how much training data 
N_train = round(len(usedset) * 0.8)
N_rest = len(usedset) - N_train
trainset, restset = random_split(usedset, [N_train, N_rest])
# how much test and validation data 
N_test = round(len(restset) * 0.5)
N_val = len(restset) - N_test
testset, valset = random_split(restset, [N_test, N_val])

# create dataloaders
BATCH_SIZE=16
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=6,pin_memory=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=BATCH_SIZE, shuffle=True, num_workers=6,pin_memory=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True, num_workers=6,pin_memory=True)

# ----- MODEL: -----
# model=models.Conv1D_VAE(x_len=24000,h_len=256,z_len=24).to(DEVICE)
model=models.AutoencoderConv(z_len=24).to(DEVICE)

# -- Training: --
LEARNRATE=1e-3
N_EPOCHS=10
trainparams={
"num_epochs": N_EPOCHS, 
"device": DEVICE,
"learnrate":LEARNRATE,
"optimizer": torch.optim.Adam(model.parameters(), LEARNRATE),
"criterion": nn.MSELoss()}

# training
start = time.time()
outputs_evol, loss_evol=train.training(model, trainloader, valloader, trainparams, 1)
end=time.time()
now=datetime.now(); dt_string = now.strftime("%d-%m-%Y--%H-%M")
print(f"training time: {(end-start)}" ) 
torch.save(model.state_dict(), projectdir + "models/trained_model_"+dt_string+".pth")


In [ ]:
# save training report
report = {
    'model'      : str(model),
    'preproc'    : PREPROC,
    'sr'         : SAMPLING_RATE,
    'start_time' : start,
    'end_time'   : end,
    'loss_evol'  : loss_evol,
    'n_epochs'   : N_EPOCHS,
    'learnrate'  : LEARNRATE,
    'batch_size' : BATCH_SIZE,
    'data_used'  : perc_data_use
    }

# the json file where the output must be stored
import json
out_file = open(f"{projectdir}models/train_report_{dt_string}_.json", "w")
json.dump(report, out_file, indent = None)
out_file.close()

loss_evol_unziped=list(zip(*loss_evol))
plt.figure(figsize=(6, 6))
train_loss=loss_evol_unziped[0][1:]
val_loss=loss_evol_unziped[1][1:]
plt.plot(train_loss,label="training loss")
plt.plot(val_loss,label="validation loss")
plt.legend()

